In [1]:
#pip install category-encoders
#!pip3 install pyforest
#pip install lightgbm

In [2]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
import pyforest
from datetime import datetime
from datetime import date, timedelta
from category_encoders import TargetEncoder
from lightgbm import LGBMRegressor

Загрузим данные:

In [3]:
data_train = pd.read_csv('train.csv')
data_train

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
0,2019-12-30,425.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,0.0,1
1,2019-12-30,525.0,22939.0,NaN,0.0,0.0,3649.0,2371.85,0.0,1.0,3
2,2019-12-30,557.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,1.0,5
3,2019-12-30,562.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,0.0,7
4,2019-12-30,637.0,22939.0,NaN,0.0,0.0,3649.0,3101.65,0.0,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...
2333098,2019-12-30,NaN,NaN,NaN,1.0,0.0,3949.0,2369.40,0.0,1.0,2337654
2333099,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,2337655
2333100,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,2337656
2333101,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,2337657


In [4]:
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub

,id,demand
0,16,1.0
1,7867,1.0
2,8667,1.0
3,14835,1.0
4,14959,1.0
...,...,...
588,203996,1.0
589,203997,1.0
590,204035,1.0
591,204037,1.0


In [5]:
ind = sample_sub['id'].tolist()
data_test = data_train[data_train['id'].isin(ind)].copy()
data_test

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
10,2019-12-30,1162.0,22939.0,NaN,0.0,0.0,3649.0,2189.40,0.0,1.0,16
7781,2019-12-30,1162.0,23162.0,NaN,0.0,0.0,3799.0,2279.40,0.0,1.0,7867
8574,2019-12-16,1162.0,23163.0,NaN,0.0,0.0,3799.0,2849.25,0.0,1.0,8667
14686,2019-12-23,798.0,23632.0,NaN,0.0,0.0,3999.0,2399.40,0.0,1.0,14835
14797,2019-12-30,1347.0,23633.0,NaN,0.0,0.0,3999.0,2999.25,0.0,1.0,14959
...,...,...,...,...,...,...,...,...,...,...,...
199484,2019-12-16,1326.0,139157.0,NaN,1.0,0.0,4899.0,2449.50,0.0,1.0,203996
199485,2019-12-09,1347.0,139157.0,NaN,0.0,0.0,4899.0,3674.25,0.0,0.0,203997
199521,2019-12-09,425.0,148040.0,NaN,0.0,0.0,3549.0,2661.75,0.0,1.0,204035
199523,2019-12-09,525.0,148040.0,NaN,0.0,0.0,3549.0,2661.75,0.0,1.0,204037


In [6]:
index_answer = data_test['id'].tolist()

In [7]:
ind = sample_sub['id'].tolist()
data_train = data_train[np.logical_not(data_train['id'].isin(ind))].copy()
data_train

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
0,2019-12-30,425.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,0.0,1
1,2019-12-30,525.0,22939.0,NaN,0.0,0.0,3649.0,2371.85,0.0,1.0,3
2,2019-12-30,557.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,1.0,5
3,2019-12-30,562.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,0.0,7
4,2019-12-30,637.0,22939.0,NaN,0.0,0.0,3649.0,3101.65,0.0,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...
2333098,2019-12-30,NaN,NaN,NaN,1.0,0.0,3949.0,2369.40,0.0,1.0,2337654
2333099,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,2337655
2333100,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,2337656
2333101,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,2337657


In [8]:
to_drop = ['id']
data_train.drop(columns=to_drop, inplace=True)
data_test.drop(columns=to_drop, inplace=True)

data_train = data_train[data_train['demand'].notna()].copy()
data_train

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
15,2019-12-02,637.0,22939.0,0.035398,1.0,0.0,3649.0,3101.65,0.0,1.0
16,2017-08-28,317.0,22939.0,1.000000,1.0,0.0,2380.0,2380.00,0.0,1.0
17,2018-03-05,355.0,22939.0,2.000000,0.0,0.0,3549.0,2306.85,0.0,1.0
18,2018-03-19,355.0,22939.0,1.013850,0.0,0.0,3549.0,2306.85,0.0,1.0
19,2018-03-26,355.0,22939.0,0.019391,0.0,0.0,3549.0,2306.85,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
284114,2019-11-11,1316.0,149472.0,1.000000,1.0,0.0,4449.0,2224.50,0.0,1.0
284115,2019-11-11,1316.0,149509.0,1.000000,1.0,0.0,3999.0,1999.50,0.0,1.0
284116,2019-11-18,1316.0,149354.0,2.000000,1.0,0.0,9799.0,4899.50,0.0,1.0
284117,2019-11-25,1316.0,25952.0,1.000000,1.0,0.0,4299.0,2149.50,0.0,1.0


In [9]:
data_train.dtypes

period_dt             object
location_id          float64
product_id           float64
demand               float64
PROMO1_FLAG          float64
PROMO2_FLAG          float64
PRICE_REGULAR        float64
PRICE_AFTER_DISC     float64
NUM_CONSULTANT       float64
AUTORIZATION_FLAG    float64
dtype: object

In [10]:
data_train.describe() 

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
count,274471.000000,274471.000000,274471.000000,272680.000000,272680.000000,267888.000000,266331.000000,272680.000000,272680.000000
mean,887.299165,60502.102488,0.418661,0.421960,0.002633,3328.855404,2221.734909,0.354148,0.871699
std,344.495491,29073.786470,0.725600,0.520525,0.051246,938.780253,681.803009,1.268765,0.334425
min,309.000000,22939.000000,0.000000,0.000000,0.000000,369.000000,4.960000,0.000000,0.000000
25%,557.000000,32990.000000,0.038356,0.000000,0.000000,2849.000000,1749.500000,0.000000,1.000000
50%,862.000000,62261.000000,0.103395,0.000000,0.000000,3499.000000,2174.250000,0.000000,1.000000
75%,1281.000000,83275.000000,0.724179,1.000000,0.000000,3849.000000,2661.750000,0.000000,1.000000
max,1380.000000,149522.000000,48.000000,3.000000,1.000000,13149.000000,11049.000000,8.000000,1.000000


In [11]:
data_test.describe() 

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
count,593.000000,593.000000,0.0,588.000000,588.0,583.000000,583.000000,588.000000,588.000000
mean,983.153457,113727.856661,NaN,0.911565,0.0,3705.756432,1894.111900,0.780612,0.996599
std,347.833437,24440.102347,NaN,0.284169,0.0,1222.999550,686.029465,1.889944,0.058271
min,317.000000,22939.000000,NaN,0.000000,0.0,579.000000,289.000000,0.000000,0.000000
25%,644.000000,101470.000000,NaN,1.000000,0.0,2750.000000,1375.000000,0.000000,1.000000
50%,1162.000000,119474.000000,NaN,1.000000,0.0,3799.000000,1899.500000,0.000000,1.000000
75%,1316.000000,130630.000000,NaN,1.000000,0.0,3949.000000,2049.500000,0.000000,1.000000
max,1380.000000,148683.000000,NaN,1.000000,0.0,13149.000000,6574.500000,11.000000,1.000000


In [12]:
data_test.drop(columns=['demand'], inplace=True)

In [13]:
data_train.nunique()

period_dt              152
location_id             41
product_id            2117
demand               16291
PROMO1_FLAG              4
PROMO2_FLAG              2
PRICE_REGULAR         2606
PRICE_AFTER_DISC     12711
NUM_CONSULTANT           9
AUTORIZATION_FLAG        2
dtype: int64

In [14]:
data_train.isna().sum()

period_dt               0
location_id             0
product_id              0
demand                  0
PROMO1_FLAG          1791
PROMO2_FLAG          1791
PRICE_REGULAR        6583
PRICE_AFTER_DISC     8140
NUM_CONSULTANT       1791
AUTORIZATION_FLAG    1791
dtype: int64

PROMO1_FLAG, PROMO2_FLAG заполним нулями, используя логику, как на семинарах, если акция не указана, скорее всего ее не было. Так же поступим с NUM_CONSULTANT. AUTORIZATION_FLAG тоже заполним 0, преполагая, что авторицазии не было. PRICE_REGULAR и PRICE_AFTER_DISC заполним средними значениями по столбцам:

In [15]:
price_reg = data_train["PRICE_REGULAR"].mean()
price_disc = data_train["PRICE_AFTER_DISC"].mean()
values = {"PROMO1_FLAG": 0, "PROMO2_FLAG": 0, "NUM_CONSULTANT": 0, "AUTORIZATION_FLAG": 0,
          "PRICE_REGULAR": price_reg, "PRICE_AFTER_DISC": price_disc}
data_train = data_train.fillna(value=values)
data_train

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
15,2019-12-02,637.0,22939.0,0.035398,1.0,0.0,3649.0,3101.65,0.0,1.0
16,2017-08-28,317.0,22939.0,1.000000,1.0,0.0,2380.0,2380.00,0.0,1.0
17,2018-03-05,355.0,22939.0,2.000000,0.0,0.0,3549.0,2306.85,0.0,1.0
18,2018-03-19,355.0,22939.0,1.013850,0.0,0.0,3549.0,2306.85,0.0,1.0
19,2018-03-26,355.0,22939.0,0.019391,0.0,0.0,3549.0,2306.85,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
284114,2019-11-11,1316.0,149472.0,1.000000,1.0,0.0,4449.0,2224.50,0.0,1.0
284115,2019-11-11,1316.0,149509.0,1.000000,1.0,0.0,3999.0,1999.50,0.0,1.0
284116,2019-11-18,1316.0,149354.0,2.000000,1.0,0.0,9799.0,4899.50,0.0,1.0
284117,2019-11-25,1316.0,25952.0,1.000000,1.0,0.0,4299.0,2149.50,0.0,1.0


In [16]:
data_train.isna().sum()

period_dt            0
location_id          0
product_id           0
demand               0
PROMO1_FLAG          0
PROMO2_FLAG          0
PRICE_REGULAR        0
PRICE_AFTER_DISC     0
NUM_CONSULTANT       0
AUTORIZATION_FLAG    0
dtype: int64

In [17]:
data_train.dtypes

period_dt             object
location_id          float64
product_id           float64
demand               float64
PROMO1_FLAG          float64
PROMO2_FLAG          float64
PRICE_REGULAR        float64
PRICE_AFTER_DISC     float64
NUM_CONSULTANT       float64
AUTORIZATION_FLAG    float64
dtype: object

In [18]:
def transform_date(df):
    df['date_'] = df['period_dt'].apply(pd.to_datetime)
    df['day_of_year'] = df['date_'].apply(lambda x : x.dayofyear)
    df['month'] = df['date_'].apply(lambda x : x.month)
    df['day_of_week'] = df['date_'].apply(lambda x : x.weekday())
    df['week_of_year'] = df['date_'].apply(lambda x : x.weekofyear)
    df['year'] = df['date_'].apply(lambda x : x.year)

transform_date(data_train)
transform_date(data_test)

In [19]:
data_train['date_'] = data_train['date_'].apply(lambda x: x.date())
data_test['date_'] = data_test['date_'].apply(lambda x: x.date())

In [20]:
data_train.dtypes

period_dt             object
location_id          float64
product_id           float64
demand               float64
PROMO1_FLAG          float64
PROMO2_FLAG          float64
PRICE_REGULAR        float64
PRICE_AFTER_DISC     float64
NUM_CONSULTANT       float64
AUTORIZATION_FLAG    float64
date_                 object
day_of_year            int64
month                  int64
day_of_week            int64
week_of_year           int64
year                   int64
dtype: object

In [21]:
data_train['demand_for_week'] = 0
for el in data_train['year'].unique().tolist():
    sup = data_train[data_train['year'] == el][['week_of_year', 'demand']].groupby(by='week_of_year').sum()
    sup = dict(zip(sup.index.tolist(), sup.demand)) 
    for key, val in sup.items():
        data_train.loc[(data_train['year'] == el) & (data_train['week_of_year'] == key), 'demand_for_week'] = val

In [22]:
data_train['peaks'] = data_train['demand_for_week'].apply(lambda x: 1 if x > 1400 else 0)

In [23]:
tmp = data_train[data_train['peaks'] == 1].copy()
peak_dates = set(tmp['date_'].tolist()) 
sorted(peak_dates)

[datetime.date(2016, 12, 26),
 datetime.date(2017, 11, 6),
 datetime.date(2017, 11, 13),
 datetime.date(2017, 11, 27),
 datetime.date(2017, 12, 25),
 datetime.date(2018, 1, 1),
 datetime.date(2018, 3, 5),
 datetime.date(2018, 11, 5),
 datetime.date(2018, 11, 26),
 datetime.date(2018, 12, 17),
 datetime.date(2018, 12, 24),
 datetime.date(2018, 12, 31),
 datetime.date(2019, 3, 4),
 datetime.date(2019, 11, 4),
 datetime.date(2019, 11, 11),
 datetime.date(2019, 11, 18),
 datetime.date(2019, 11, 25),
 datetime.date(2019, 12, 2)]

In [24]:
data_test['peaks'] = 1

In [25]:
reg_mean = data_train[['location_id', 'demand']].groupby(by='location_id').mean().sort_values(by='demand')
reg_mean

,demand
location_id,
533.0,0.218085
504.0,0.223932
1173.0,0.234142
540.0,0.234395
562.0,0.235032
...,...
1347.0,0.686151
1380.0,0.771235
1316.0,0.881877


In [26]:
def get_type_reg(x):
    if x >= 0.70:
        return 5
    elif x >= 0.50:
        return 3
    return 1
        
reg_mean['type'] = reg_mean['demand'].apply(get_type_reg)

regs = dict(zip(reg_mean.index.tolist(), reg_mean['type'].tolist()))
data_train['reg_type'] = data_train['location_id'].apply(lambda x: regs[x] if x in regs else 0)
data_test['reg_type'] = data_test['location_id'].apply(lambda x: regs[x] if x in regs else 0)

In [27]:
sup = data_train[['product_id', 'location_id', 'date_']].groupby(by=['product_id', 'location_id']).min()
sup = sup.sort_values('date_')
sup

,,date_
product_id,location_id,
62406.0,1281.0,2016-12-19
62003.0,657.0,2016-12-19
40720.0,309.0,2016-12-19
32680.0,644.0,2016-12-19
37230.0,557.0,2016-12-19
...,...,...
104488.0,1380.0,2019-12-02
79893.0,1328.0,2019-12-02
130629.0,355.0,2019-12-02


In [28]:
sup = data_train[['product_id', 'location_id', 'date_']].groupby(by=['product_id', 'location_id']).min()
sup = sup.sort_values('date_')

In [29]:
sup = sup[sup['date_'] >= date(2017, 1, 9)].copy()

In [30]:
time_dict = dict(zip(sup.index.tolist(), zip(sup.date_.tolist(), (sup.date_ + timedelta(days=24)).tolist())))
def func(x, sup):
    if (x.product_id, x.location_id) in sup:
        el = sup[(x.product_id, x.location_id)]
        if el[0] <= x.date_ <= el[1]:
            return 1
        else:
            return 0
    return 0

data_train['take'] = 0
data_train['take'] = data_train.apply(func, args=[time_dict], axis=1)

In [31]:
data_train = data_train[data_train['take'] == 1].copy()

In [32]:
cats = ['product_id', 'location_id']

to_del = ['demand_for_week', 'period_dt', 'date_']
data_train.drop(columns=to_del, inplace=True)

In [33]:
data_test.drop(columns=['period_dt'], inplace=True)
data_test.drop(columns=['date_'], inplace=True)
data_train.drop(columns=['take'], inplace=True)

target = data_train['demand']
data_train.drop(columns=['demand'], inplace=True)

In [34]:
data_train.columns

Index(['location_id', 'product_id', 'PROMO1_FLAG', 'PROMO2_FLAG',
       'PRICE_REGULAR', 'PRICE_AFTER_DISC', 'NUM_CONSULTANT',
       'AUTORIZATION_FLAG', 'day_of_year', 'month', 'day_of_week',
       'week_of_year', 'year', 'peaks', 'reg_type'],
      dtype='object')

In [35]:
encoder = TargetEncoder(cols=cats)
data_train = encoder.fit_transform(data_train, target).copy()
data_test = encoder.transform(data_test).copy()

C:\Users\kulak\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\kulak\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [36]:
regressor =LGBMRegressor().fit(data_train, target)
y_pred = regressor.predict(data_test)

In [37]:
answer = dict(zip(index_answer, y_pred.tolist()))
submission['demand'] = submission['id'].apply(lambda x: answer[x])

In [38]:
submission

,id,demand
0,16,0.751708
1,7867,0.578480
2,8667,0.932162
3,14835,0.674509
4,14959,0.849199
...,...,...
588,203996,1.129344
589,203997,1.083201
590,204035,5.968806
591,204037,4.917674


In [39]:
submission.to_csv('submission.csv', index=False)